PART I: extracting and tokenizing the text

In [1]:
import json
import pandas as pd
from ast import literal_eval
import glob
import csv
import copy
import os

Load data from fsm_data folder. Currently, the manual analysis do have the game DisYouCatchTheBall, TheDice, and Tangram so we remove them



> Note: the dir variable may differ in the local folder




In [8]:
data = []
dir = '../fsm_data/*'
files = sorted(glob.glob(dir, recursive=False))
for single_file in files:
  if ((single_file=='/content/fsm_data/DidYouCatchTheBall_US.json') or (single_file == '/content/fsm_data/TheDice_US.json') or (single_file == '/content/fsm_data/TangramsRace.json')):
    files.remove(single_file)
  if (single_file=='/content/fsm_data/TheDice_US.json'):
    files.remove(single_file)
  print(single_file)
#print(files)
for single_file in files:
    with open(single_file, 'r') as f:
        json_file = json.load(f)
        new_string = json.dumps(json_file, indent = 2)
        # print(new_string)
        data.append(json_file)

Extract all of the text in the game. The considered text are from 'displayText' attribute in the json file

In [3]:
c = []
for single_file in data:
    temp = single_file['states']
    d = []
    #print('------------------------------')
    for i in temp:
        att = list(i.keys())
        for key in att:
            if (key == 'displayText'):
                s = ''
                text = list(i[key].keys())
                for j in text:
                    s += i[key][j]
                if (not (s == '')):
                    d.append(s)
    c.append(d)

In [9]:
# pip install transformers

In [10]:
# pip install tensorflow

In [11]:
import tensorflow as tf
from tensorflow import keras

In [12]:
# pip install pytorch-pretrained-bert pytorch-nlp

In [13]:
import torch
from transformers import BertTokenizer, BertModel
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import io
import numpy as np
import matplotlib.pyplot as plt
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [14]:
import nltk
nltk.download('all')
from nltk import tokenize

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\vtminh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\vtminh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\vtminh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\vtminh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     C:\Users\vtminh\AppData\Roaming\nltk_data...
[nltk_data] 

The following two cells is to separate the whole text in each game into sentences and add [CLS] and [SEP] to start and end of the sentences. This is because BERT requires these two tokens to work.

In [15]:
sentences = []
for i in c:
    sentences_i = []
    for j in i:
        p = nltk.sent_tokenize(j) 
        sentences_i.append(p)
    sentences.append(sentences_i)

In [17]:
sentences_tuned = []
index = 0
for i in sentences:
    sentences_tuned.append([])
    for j in i:
        for k in j:
            temp = "[CLS] " + k + " [SEP]"
            sentences_tuned[index].append(temp)
    index += 1

# print(sentences_tuned[0])

Tokenize each words (needed for BERT)

In [18]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenized_paras = []
for para in sentences_tuned:
    tokenized_texts = [tokenizer.tokenize(sent) for sent in para]
    tokenized_paras.append(tokenized_texts)
# print(tokenized_paras[0])

Convert the tokens into numerical ids. The input_ids stores the text as numerical data for each token. 
we also have to pad_sequences, which means that making all sentences turn to the same length (pad sentences with 0 - inserting 0 into input_ids)

The input_ids in this case is a 2D array where each 1D store the ids of the text each games.

The attention_mask and seq_mask is for the program to notice specific words (it will likely ignore unmeaningful words)

In [19]:
# # Set the maximum sequence length. 
MAX_LEN = 128
# Pad our input tokens
input_ids = []
seq_mask = []
for para in tokenized_paras:
  # for i in range(0, len(para)-1):
  input_ids.append(pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in para], maxlen=MAX_LEN, dtype="long", truncating="post", padding="post"))
# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
  #for i in range(0, len(input_ids)):
  i = len(input_ids)-1
  input_ids[i] = [tokenizer.convert_tokens_to_ids(x) for x in para]
  input_ids[i] = pad_sequences(input_ids[i], maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

#print(input_ids)

# Create attention masks
attention_mask = []
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for j in input_ids:  
  for seq in j:
    seq_mask.append([float(i>0) for i in seq])
    attention_mask.append(seq_mask)
  attention_masks.append(attention_mask)


PART II: Creating dataframe

Loading csv files from manual_data folder
"The Mortal Gorilla" game do not contained in json file, so we have to drop it.

We use the pandas DataFrame to load the csv file into. 
We also drop/remove some of the attributes that we considered to be unrelated to text specifically.

array x store the names of the remaining attributes.
labels is an array that store DataFrames of the csv files.

In [20]:
dir_csv = '../manual-analysis/*'
csv_files = sorted(glob.glob(dir_csv, recursive=False))
#csv_files.remove('/content/manual_data/Mortal Gorilla - Sheet1.csv')
# print(len(csv_files))
# print(len(files))
df_list = (pd.read_csv(file) for file in csv_files)
labels = list(df_list)
for y in labels:
  y.drop(columns=["NAME","Day","Presence of Teams", "Team Dynamics No Teams", "Team Size", "Number of Teams", "Team Dynamics Between Teams", "Team Dynamics Within Teams", "Drawing Components [Rules]", "Drawing Components [Physical Objects]", "Drawing Components [Physical Space]", "Drawing Components [Timing]", "Drawing Components [Physicality]", "FSMD Components [Rules]", "FSMD Components [Physical Objects]", "FSMD Components [Physical Space]", "FSMD Components [Timing]", "FSMD Components [Physicality]", "Presence of Finite State Machine Diagram", "Output State Representation", "Transition State Representation", "Finite State Machine Diagram Consistency with Specified Rules", "State Consistency (Boxes)", "Transition Consistency (Arrows)", "Finite State Machine Diagram Completion", "States/Boxes", "Transitions/Arrows", "Numbered States", "Evidence of Programming Language Knowledge [Arrow(s) that loop to a previous state]"], inplace = True)

x = list(labels[0])
print(x)



KeyError: "['NAME', 'Day', 'Presence of Teams', 'Team Dynamics No Teams', 'Team Size', 'Number of Teams', 'Team Dynamics Between Teams', 'Team Dynamics Within Teams', 'Drawing Components [Rules]', 'Drawing Components [Physical Objects]', 'Drawing Components [Physical Space]', 'Drawing Components [Timing]', 'Drawing Components [Physicality]', 'FSMD Components [Rules]', 'FSMD Components [Physical Objects]', 'FSMD Components [Physical Space]', 'FSMD Components [Timing]', 'FSMD Components [Physicality]', 'Presence of Finite State Machine Diagram', 'Output State Representation', 'Transition State Representation', 'Finite State Machine Diagram Consistency with Specified Rules', 'State Consistency (Boxes)', 'Transition Consistency (Arrows)', 'Finite State Machine Diagram Completion', 'States/Boxes', 'Transitions/Arrows', 'Numbered States', 'Evidence of Programming Language Knowledge [Arrow(s) that loop to a previous state]'] not found in axis"

Divide the data into train and test part. The train_test_split divide the both the input_ids and labels into two train part and test part (test_size in this case is 0.3, meaning that we are testing on 30% of the data we have)

We then convert all of the data into tensor (for the model to run).

Currently, we only keep the numerical part of the Dataframe so that it can convert to tensor 


In [21]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, test_size=0.3)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids, test_size=0.3)

#print(train_labels)

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)

#list_of_tensors = [np.array(df) for df in train_labels]
# print(list_of_tensors)
#torch.tensor(np.stack(list_of_tensors))
tempf = []
for u in train_labels:
  #torch_tensor = [torch.tensor(u[k].values) for k in x]
  #temp.append(torch.stack(torch_tensor))
  tempt = []
  for k in x:
    t = (u[k].values)[0]
    tempt.append(t)
    #print(t)
  tempf.append(tempt)
  #print(u['Game Descriptor'].values)
train_labels = torch.tensor(tempf)
#print(train_labels)
###########################################################################
# torch_tensor = [torch.tensor(validation_labels[0][k].values) for k in x]
# validation_labels = torch.stach(torch_tensor)
#validation_labels = torch.tensor(validation_labels)

tempf = []
for u in validation_labels:
  tempt = []
  for k in x:
    t = (u[k].values)[0]
    tempt.append(t)
  tempf.append(tempt)
validation_labels = torch.tensor(tempf)

train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)
 

ValueError: Found input variables with inconsistent numbers of samples: [0, 8]

In [22]:
batch_size = 3

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


NameError: name 'train_inputs' is not defined

In [23]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 55)
#model.cuda()

100%|██████████| 407873900/407873900 [00:33<00:00, 12076618.38B/s]


In [24]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

SystemError: GPU device not found

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

AssertionError: Torch not compiled with CUDA enabled

In [26]:
# print(len(input_ids))
# print(len(train_inputs))
# print(len(train_labels))
# print(len(validation_labels))
# print(len(validation_inputs))
# print(train_inputs.shape)
# print(train_labels.shape)
# print(train_inputs)

In [27]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=2e-5,
                     warmup=.1)

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)
  
# Store our loss and accuracy for plotting
train_loss_set = []
# Number of training epochs 
epochs = 4

# BERT training loop
for _ in trange(epochs, desc="Epoch"):  
  
  ## TRAINING
  
  # Set our model to training mode
  model.train()  
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch

    print(b_input_ids.shape)
    print(len(input_ids))
    print((b_input_mask.shape))
    print((b_labels.shape))

    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass
    loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    train_loss_set.append(loss.item())    
    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1
  print("Train loss: {}".format(tr_loss/nb_tr_steps))
       
  ## VALIDATION

  # Put model in evaluation mode
  model.eval()
  # Tracking variables 
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0
  # Evaluate data for one epoch
  for batch in validation_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)    
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)    
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1
  print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

# plot training performance
plt.figure(figsize=(15,8))
plt.title("Training loss")
plt.xlabel("Batch")
plt.ylabel("Loss")
plt.plot(train_loss_set)
plt.show()

t_total value of -1 results in schedule not being applied
Epoch:   0%|          | 0/4 [00:00<?, ?it/s]


NameError: name 'train_dataloader' is not defined